## TF2.0版本DCGAN实现

In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

### 1. 数据导入与预处理

In [4]:
(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()
# reshape data to 3-d and change data type to float32
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5      # Normalize the images to [-1, 1]
BUFFER_SIZE = 60000
BATCH_SIZE = 256
# Batch and shuffle the data
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
train_dataset

<BatchDataset shapes: (None, 28, 28, 1), types: tf.float32>

### 2. 构建生成器和判别器模型

In [13]:
def generator_model():
    model = tf.keras.Sequential([
        # generator最开始接收到的是一个长度为100的一维向量，因此先将其与Dense层进行连接
        # 为了方便后续反卷积，因此将其接收的units设定如下，这样可以后续以[7, 7]的shape进行反卷积
        tf.keras.layers.Dense(units=7*7*256, 
                              input_shape=(100,), 
                              use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LeakyReLU(),

        tf.keras.layers.Reshape((7, 7, 256)), # [7, 7, 256]

        tf.keras.layers.Conv2DTranspose(filters=128, kernel_size=(5, 5), 
                                        strides=(1, 1), 
                                        padding='same',
                                        use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LeakyReLU(),          # [7, 7, 128]

        tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=(5, 5), 
                                        strides=(2, 2), 
                                        padding='same', 
                                        use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LeakyReLU(),          # [14, 14, 64]

        tf.keras.layers.Conv2DTranspose(filters=1, kernel_size=(5, 5), 
                                        strides=(2, 2), 
                                        padding='same', 
                                        use_bias=False, 
                                        activation='tanh') # [28, 28, 1]         
                                        ])
    return model

In [14]:
def discriminator_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(filters=64, kernel_size=(5, 5),
                               strides=(2, 2), padding="same",
                               input_shape=(28, 28, 1)),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.Dropout(rate=0.3), # 避免D别训练的太好了

        tf.keras.layers.Conv2D(filters=128, kernel_size=(5, 5),
                               strides=(2, 2), padding="same"),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.Dropout(rate=0.3), # 避免D别训练的太好了

        tf.keras.layers.Conv2D(filters=256, kernel_size=(5, 5),
                               strides=(2, 2), padding="same"),
        tf.keras.layers.LeakyReLU(),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=1)
    ])
    return model

### 3. 定义损失函数以及训练函数

In [ ]:
# https://blog.csdn.net/qq_36758914/article/details/104455898
# https://blog.csdn.net/claroja/article/details/98586682
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(y_true=tf.ones_like(real_output), y_pred=real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

EPOCHS = 100
noise_dim = 100
num_examples_to_generate = 16

seed = tf.random.normal([num_examples_to_generate, noise_dim])

generator = generator_model()
discriminator = discriminator_model()

In [ ]:
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [ ]:
def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
    predictions = model(test_input, training=False)

    fig = plt.figure(figsize=(4, 4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow((predictions[i, :, :, 0] + 1)/2, cmap='gray')
        plt.axis('off')

    plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()

In [ ]:
def train(dataset, epochs):
    for epoch in range(epochs):
        for image_batch in dataset:
            train_step(image_batch)
            print('.', end='')
        print()
        generate_and_save_images(generator, epoch + 1, seed)
    generate_and_save_images(generator, epochs, seed)

In [ ]:
# 训练模型
train(train_dataset, EPOCHS)